# Notebook Examples for Chapter 1
# Images, Arrays and Matrices
There are many  Earth observation satellite-based sensors, both active and passive, currently in orbit or planned for the near future.
Laying the mathematical foundation for the image analysis procedures and algorithms forming the substance of the text,
Chapter 1 begins with a short description of typical remote sensing imagery in the  optical/infrared  and synthetic aperture radar categories, together with their representations as digital arrays.  The multispectral ASTER system and the TerraSAR-X synthetic aperture radar satellite are chosen as illustrations.  Then some basic concepts of linear algebra of vectors and matrices are introduced, namely linear dependence,  eigenvalues and eigenvectors, singular value  decomposition and finding minima and maxima using Lagrange multipliers. The latter is illustrated with the principal components analysis of a multispectral image.

In [ ]:
# Requirements
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
import numpy as np
from IPython.display import Image

%matplotlib inline

import ee
#ee.Authenticate()
#ee.Initialize()

In [ ]:
!gdalinfo --version

In [ ]:
!python -V

## 1.1 Multispectral satellite images

In [ ]:
run scripts/ex1_1 imagery/AST_20070501.tif  3 

## 1.2 Synthetic aperture radar images

In [ ]:
minlon = 6.31
minlat = 50.83
maxlon = 6.58
maxlat = 50.95

rect = ee.Geometry.Rectangle([minlon,minlat,maxlon,maxlat])
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filterBounds(rect) \
  .filterDate(ee.Date('2017-05-01'), ee.Date('2017-06-01')) \
  .filter(ee.Filter.eq('transmitterReceiverPolarisation', 
                                              ['VV','VH'])) \
  .filter(ee.Filter.eq('resolution_meters', 10)) \
  .filter(ee.Filter.eq('instrumentMode', 'IW')) 
    
image = ee.Image(collection.first()).clip(rect)                        
url = image.select('VV').getThumbURL({'min':-20,'max':0})
Image(url=url)

## 1.3 Linear algebra of vectors and matrices
### 1.3.1 Elementary properties
### 1.3.2 Square matrices
### 1.3.3 Singular matrices
### 1.3.4 Symmetric, positive definite matrices

In [ ]:
gdal.AllRegister()
infile = 'imagery/AST_20070501.tif'                
inDataset = gdal.Open(infile,GA_ReadOnly)     
cols = inDataset.RasterXSize
rows = inDataset.RasterYSize    

#  data matrix
G = np.zeros((rows*cols,3))                                   
for b in range(3):
    band = inDataset.GetRasterBand(b+1)
    tmp = band.ReadAsArray(0,0,cols,rows).ravel()
    G[:,b] = tmp - np.mean(tmp)
    
#  covariance matrix
C = np.mat(G).T*np.mat(G)/(cols*rows-1)  
C

### 1.3.5 Linear dependence and vector spaces
### 1.3.6 Eigenvalues and eigenvectors

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(C)
print(eigenvalues)
print(eigenvectors)

In [ ]:
U = eigenvectors
print(U.T*U)

### 1.3.7 Singular value decomposition

In [ ]:
b = np.mat([1,2,3])
# an almost singular matrix
A = b.T*b + np.random.rand(3,3)*0.001
# a symmetric almost singular matrix
A = A + A.T
print('determinant: %f'%np.linalg.det(A))
# singular value decomposition
U,Lambda,V = np.linalg.svd(A)
print('Lambda = %s'%str(Lambda))
print('U = %s'%str(U))
print('V = %s'%str(V))

### 1.3.8 Tensors

In [ ]:
import tensorflow as tf
data = [[1., 2.],[3., 4.]]
tf_data = tf.constant(data)
np_array = np.array(tf_data)
tf_np = tf.constant(np_array)
print(tf_np)

## 1.4 Finding minima and maxima

In [ ]:
%run scripts/ex1_2 imagery/AST_20070501.tif imagery/ex1_2.tif

In [ ]:
%run scripts/dispms -f 'imagery/ex1_2.tif' -p [1,2,3] -e 4 \
#-s '~/LaTeX/new projects/CRC5/Chapter1/fig1_8.eps'